In [ ]:
import tensorflow as tf
import numpy

In [ ]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
strategy = tf.distribute.TPUStrategy(cluster_resolver)
print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
from stable_diffusion_tf.stable_diffusion import Text2Image
from PIL import Image

In [ ]:
with strategy.scope():  # Just a 1-line change
    generator = Text2Image( 
      img_height=512,
      img_width=512,
)

In [ ]:
imgs = generator.generate(
    "A beautiful sunset, matte painting and 4k",
    num_steps=50,
    batch_size=8
)

In [ ]:
i = 0
for img in imgs:
    if numpy.average(img) != 124:
        pil_img = Image.fromarray(img)
        pil_img.save(f'{i}.png')
        i += 1

In [ ]:
from PIL import Image

In [ ]:
!ls

In [ ]:
myimg = Image.open('2.png')
myimg.show()

In [ ]:
1

In [6]:
import discord 
from discord.ext import commands
import typing
import functools
import asyncio

import tensorflow as tf
import numpy
from stable_diffusion_tf.stable_diffusion import Text2Image
from PIL import Image

In [79]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
strategy = tf.distribute.TPUStrategy(cluster_resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2022-10-02 02:54:05.648735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-02 02:54:09.035516: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x25a0300 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
2022-10-02 02:54:09.035548: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): TPU, 2a886c8
2022-10-02 02:54:09.035555: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (1): TPU, 2a886c8
2022-10-02 02:54:09.035560: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (2): TPU, 2a886c8
2022-10-02 02:54:09.035565: I tensorflow/compiler/xla/service/service.cc:178]   

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [9]:
with strategy.scope(): 
    generator = Text2Image( 
        img_height=512,
        img_width=512,
)

2022-10-02 02:54:21.990329: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:263] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-10-02 02:54:22.012870: I tensorflow/compiler/jit/xla_compilation_cache.cc:478] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-10-02 02:54:22.250686: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. RandomUniform


In [68]:
bot = commands.Bot(command_prefix="/", intents=discord.Intents.all(), application_id="992210335715037244")

@bot.event
async def on_ready():
    print("bot is online")

def make_embed(images, image_num, prompt):
    image = Image.fromarray(images[image_num])
    image.save("image.png")
    embed=discord.Embed(title=f"{prompt} ({image_num+1}/{len(images)})")
    file = discord.File("image.png", filename=f"image-{image_num}.png")
    embed.set_image(url=f"attachment://image-{image_num}.png")
    return file, embed

class Menu(discord.ui.View):
    def __init__(self, images, prompt):
        super().__init__()
        self.images = images
        self.number = 0
        self.prompt = prompt
        self.change_button_state("disable", "left")
    
    @discord.ui.button(label="⬅️", style=discord.ButtonStyle.grey, custom_id="left")
    async def left(self, interaction, button):
        self.number -= 1
        if self.number == 0:
            self.change_button_state("disable", "left")
        self.change_button_state("enable", "right")
        file, embed = make_embed(self.images, self.number, self.prompt)
        await interaction.response.edit_message(embed=embed, attachments=[file], view=self)

    
    @discord.ui.button(label="➡️", style=discord.ButtonStyle.grey, custom_id="right")
    async def right(self, interaction, button):
        self.number += 1
        if self.number == len(self.images)-1:
            self.change_button_state("disable", "right")
        self.change_button_state("enable", "left")
        file, embed = make_embed(self.images, self.number, self.prompt)
        await interaction.response.edit_message(embed=embed, attachments=[file], view=self)
    
    def change_button_state(self, operation, button_id): #operation is "enable" or "disable"
        for button in self.children:
            if button.custom_id == button_id: 
                if operation == "enable":
                    button.disabled = False
                elif operation == "disable":
                    button.disabled = True    

@bot.command()
async def prompt(ctx):
    SHIKHAR_ID = 711351178146873344
    if not ctx.message.guild and ctx.message.author.id != SHIKHAR_ID:
        await ctx.message.reply("Don't use this bot in DM's, use it in the server please!")
        return

    #await ctx.defer(ephemeral=True)
    prompt = ctx.message.content[8:]
    images = await gen_images(prompt)
    view = Menu(images, prompt)
    file, embed = make_embed(images, 0, prompt)
    await ctx.reply(embed=embed, file=file, view=view)

async def gen_images(prompt):   
    images = generator.generate(
        prompt,
        num_steps=50,
        batch_size=8
    )
    good_images = []
    i = 0
    for img in images:
        if numpy.average(img) != 124:
            pil_img = Image.fromarray(img)
            pil_img.save(f'{i}.png')
            good_images.append(img)
            i += 1
    print("saved images")
    return good_images

In [84]:
async def main():
    await bot.load_extension('prompt')
    await bot.run('MTAxMzIzMjQ5ODI0OTA0Mzk2OQ.GaUFlM.PVK86rFRcGOVnSU-ckP_RzqG99f6LwNHmQrZ1k')
asyncio.run(main())

[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: logging in using static token
[2022-10-02 07:08:14] [INFO    ] discord.client: log

RuntimeError: Session is closed

In [ ]:
bot = commands.Bot(command_prefix="/", intents=discord.Intents.all(), application_id="1013232498249043969")

@bot.event
async def on_ready():
    print("Bot is online")

async def main():
    await bot.load_extension('prompt')

asyncio.run(main())
bot.run('MTAxMzIzMjQ5ODI0OTA0Mzk2OQ.GaUFlM.PVK86rFRcGOVnSU-ckP_RzqG99f6LwNHmQrZ1k')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: local


INFO:tensorflow:Initializing the TPU system: local


INFO:tensorflow:Finished initializing TPU system.


...

[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: logging in using static token
[2022-10-02 09:19:40] [INFO    ] discord.client: log

Generator loaded


[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: ddea29fc1a639c72ebc908f67160e2ab).
[2022-10-02 09:19:40] [INFO    ] discord.

Bot is online
Prompt cog loaded


2022-10-02 09:20:27.322269: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_2"
op: "TensorDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorDataset:199215"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
        dim {
          size: 77
        }
      }
      shape {
        dim {
          size: 8
        }
        dim {
          size: 77
        }
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        ty